### this notebook is a copy of train.py 

In [1]:
from rl.callbacks import ModelIntervalCheckpoint, FileLogger
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Concatenate
from gym import spaces
import numpy as np
from PIL import Image
from configparser import ConfigParser
import os
from os.path import join, pardir, exists

from gym_airsim.airsim_car_env import AirSimCarEnv

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

class AirSimCarProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        # We could perform this processing step in `process_observation`. In this case, however,
        # we would need to store a `float32` array instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

config = ConfigParser()
config.read('config.ini')
num_actions = int(config['car_agent']['actions'])
                    
WINDOW_LENGTH = 4
INPUT_SHAPE = (84, 84)

env = AirSimCarEnv()
np.random.seed(123)

# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_actions))
model.add(Activation('linear'))
print(model.summary())

log_dir = 'logs'
if not exists(log_dir):
    os.makedirs(log_dir)

memory = SequentialMemory(limit=50000, window_length=WINDOW_LENGTH)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),  attr='eps', value_max=1., 
                              value_min=.1, value_test=.05, nb_steps=1000000)
processor = AirSimCarProcessor()

dqn = DQNAgent(model=model, nb_actions=num_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.0001), metrics=['mae'])

def build_callbacks(env_name):
    log_dir = 'logs'
    if not exists(log_dir):
        os.makedirs(log_dir)
    
    checkpoint_weights_filename = join(log_dir, 'dqn_' + env_name + '_weights_{step}.h5f')
    log_filename = join(log_dir,'dqn_{}_log.json'.format(env_name))
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=25000)]
    callbacks += [FileLogger(log_filename, interval=100)]
    return callbacks

callbacks = build_callbacks('AirSimCarRL')

dqn.fit(env, nb_steps=2000000,
        visualize=False,
        verbose=2,
        callbacks=callbacks)

# def build_callbacks_cont(env_name):
#     checkpoint_weights_filename = join(log_dir, 'dqn_' + env_name + '_weights_500000_add_{step}.h5f')
#     log_filename = join(log_dir,'dqn_{}_log.json'.format(env_name))
#     callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=25000)]
#     callbacks += [FileLogger(log_filename, interval=100)]
#     return callbacks

# callbacks_cont = build_callbacks_cont('AirSimCarRL')

# dqn = DQNAgent(model=model, nb_actions=num_actions, policy=policy, memory=memory,
#                processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
#                train_interval=4, delta_clip=1.)
# dqn.compile(Adam(lr=.0001), metrics=['mae'])

# weights_filename = join(log_dir,'dqn_{}_weights_{}.h5f'.format('AirSimCarRL', 500000))
# dqn.load_weights(weights_filename)

# dqn.fit(env, nb_steps=2000000,
#         visualize=False,
#         verbose=2,
#         callbacks=callbacks_cont)

Using TensorFlow backend.
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\sit

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

['WayPoint0', 'WayPoint1', 'WayPoint2', 'WayPoint3', 'WayPoint4', 'WayPoint5', 'WayPoint6', 'WayPoint7', 'WayPoint8', 'WayPoint9']



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_____________________________________________________

    5311/2000000: episode: 17, duration: 7.562s, episode steps: 273, steps per second: 36, episode reward: 113.400, mean reward: 0.415 [-1.000, 0.500], mean action: 2.945 [0.000, 6.000], mean observation: 172.384 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    5627/2000000: episode: 18, duration: 9.100s, episode steps: 316, steps per second: 35, episode reward: 179.200, mean reward: 0.567 [-1.000, 1.000], mean action: 3.035 [0.000, 6.000], mean observation: 171.963 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    5910/2000000: episode: 19, duration: 7.723s, episode steps: 283, steps per second: 37, episode reward: 105.200, mean reward: 0.372 [-1.000, 0.500], mean action: 2.979 [0.000, 6.000], mean observation: 172.880 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    6220/2000000: episode: 20, duration: 8.901s, episode steps: 310, steps per second: 35, episode reward: 183.700, mean rew

   14241/2000000: episode: 46, duration: 9.432s, episode steps: 316, steps per second: 34, episode reward: 157.000, mean reward: 0.497 [-1.000, 1.000], mean action: 3.051 [0.000, 6.000], mean observation: 172.107 [23.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   14552/2000000: episode: 47, duration: 8.628s, episode steps: 311, steps per second: 36, episode reward: 116.000, mean reward: 0.373 [-1.000, 0.500], mean action: 3.000 [0.000, 6.000], mean observation: 172.861 [22.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   14870/2000000: episode: 48, duration: 9.564s, episode steps: 318, steps per second: 33, episode reward: 174.900, mean reward: 0.550 [-1.000, 1.000], mean action: 3.019 [0.000, 6.000], mean observation: 172.196 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   15179/2000000: episode: 49, duration: 8.869s, episode steps: 309, steps per second: 35, episode reward: 143.800, mean rew

   23493/2000000: episode: 75, duration: 8.305s, episode steps: 296, steps per second: 36, episode reward: 121.300, mean reward: 0.410 [-1.000, 0.500], mean action: 2.889 [0.000, 6.000], mean observation: 171.197 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   23801/2000000: episode: 76, duration: 8.788s, episode steps: 308, steps per second: 35, episode reward: 131.000, mean reward: 0.425 [-1.000, 1.000], mean action: 3.042 [0.000, 6.000], mean observation: 171.047 [22.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   24090/2000000: episode: 77, duration: 8.028s, episode steps: 289, steps per second: 36, episode reward: 120.600, mean reward: 0.417 [-1.000, 0.500], mean action: 2.837 [0.000, 6.000], mean observation: 170.951 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   24400/2000000: episode: 78, duration: 8.927s, episode steps: 310, steps per second: 35, episode reward: 138.600, mean rew

...

 1261370/2000000: episode: 3930, duration: 20.568s, episode steps: 481, steps per second: 23, episode reward: 395.500, mean reward: 0.822 [0.500, 1.000], mean action: 3.613 [0.000, 6.000], mean observation: 165.857 [2.000, 255.000], loss: 0.087200, mean_absolute_error: 50.035563, mean_q: 58.515601, mean_eps: 0.100000
 1261842/2000000: episode: 3931, duration: 20.458s, episode steps: 472, steps per second: 23, episode reward: 444.000, mean reward: 0.941 [0.500, 1.000], mean action: 3.489 [0.000, 6.000], mean observation: 166.581 [25.000, 255.000], loss: 0.074870, mean_absolute_error: 50.168520, mean_q: 58.651467, mean_eps: 0.100000
 1262316/2000000: episode: 3932, duration: 20.362s, episode steps: 474, steps per second: 23, episode reward: 443.500, mean reward: 0.936 [0.500, 1.000], mean action: 3.407 [0.000, 6.000], mean observation: 166.276 [8.000, 255.000], loss: 0.066805, mean_absolute_error: 50.302571, mean_q: 58.803198, mean_eps: 0.100000
 1262790/2000000: episode: 3933, duration:

 1273639/2000000: episode: 3956, duration: 20.506s, episode steps: 460, steps per second: 22, episode reward: 398.000, mean reward: 0.865 [0.500, 1.000], mean action: 3.770 [0.000, 6.000], mean observation: 165.457 [14.000, 255.000], loss: 0.075574, mean_absolute_error: 50.395834, mean_q: 58.928369, mean_eps: 0.100000
 1274103/2000000: episode: 3957, duration: 20.578s, episode steps: 464, steps per second: 23, episode reward: 422.500, mean reward: 0.911 [0.500, 1.000], mean action: 3.459 [0.000, 6.000], mean observation: 165.864 [21.000, 255.000], loss: 0.068494, mean_absolute_error: 50.242230, mean_q: 58.733312, mean_eps: 0.100000
 1274582/2000000: episode: 3958, duration: 20.463s, episode steps: 479, steps per second: 23, episode reward: 422.500, mean reward: 0.882 [0.500, 1.000], mean action: 3.505 [0.000, 6.000], mean observation: 166.431 [18.000, 255.000], loss: 0.065844, mean_absolute_error: 50.460813, mean_q: 58.989966, mean_eps: 0.100000
 1275063/2000000: episode: 3959, duratio

 1285790/2000000: episode: 3982, duration: 20.513s, episode steps: 476, steps per second: 23, episode reward: 423.500, mean reward: 0.890 [0.500, 1.000], mean action: 3.576 [0.000, 6.000], mean observation: 166.297 [13.000, 255.000], loss: 0.073858, mean_absolute_error: 50.552353, mean_q: 59.100603, mean_eps: 0.100000
 1286270/2000000: episode: 3983, duration: 20.514s, episode steps: 480, steps per second: 23, episode reward: 423.500, mean reward: 0.882 [0.500, 1.000], mean action: 3.413 [0.000, 6.000], mean observation: 166.101 [17.000, 255.000], loss: 0.074359, mean_absolute_error: 50.338623, mean_q: 58.851758, mean_eps: 0.100000
 1286743/2000000: episode: 3984, duration: 20.529s, episode steps: 473, steps per second: 23, episode reward: 413.000, mean reward: 0.873 [0.500, 1.000], mean action: 3.643 [0.000, 6.000], mean observation: 165.846 [0.000, 255.000], loss: 0.074604, mean_absolute_error: 50.840702, mean_q: 59.428419, mean_eps: 0.100000
 1287220/2000000: episode: 3985, duration

 1298128/2000000: episode: 4008, duration: 20.632s, episode steps: 474, steps per second: 23, episode reward: 421.500, mean reward: 0.889 [0.500, 1.000], mean action: 3.435 [0.000, 6.000], mean observation: 166.166 [16.000, 255.000], loss: 0.080359, mean_absolute_error: 50.615182, mean_q: 59.193546, mean_eps: 0.100000
 1298608/2000000: episode: 4009, duration: 20.561s, episode steps: 480, steps per second: 23, episode reward: 371.100, mean reward: 0.773 [0.100, 1.000], mean action: 3.621 [0.000, 6.000], mean observation: 165.823 [0.000, 255.000], loss: 0.078078, mean_absolute_error: 50.689906, mean_q: 59.281361, mean_eps: 0.100000
 1299079/2000000: episode: 4010, duration: 20.539s, episode steps: 471, steps per second: 23, episode reward: 413.000, mean reward: 0.877 [0.500, 1.000], mean action: 3.711 [0.000, 6.000], mean observation: 165.853 [13.000, 255.000], loss: 0.083956, mean_absolute_error: 50.621022, mean_q: 59.180341, mean_eps: 0.100000
 1299550/2000000: episode: 4011, duration

 1310417/2000000: episode: 4034, duration: 20.524s, episode steps: 472, steps per second: 23, episode reward: 439.000, mean reward: 0.930 [0.500, 1.000], mean action: 3.487 [0.000, 6.000], mean observation: 166.046 [17.000, 255.000], loss: 0.085601, mean_absolute_error: 51.528969, mean_q: 60.241542, mean_eps: 0.100000
 1310894/2000000: episode: 4035, duration: 20.443s, episode steps: 477, steps per second: 23, episode reward: 440.000, mean reward: 0.922 [0.500, 1.000], mean action: 3.730 [0.000, 6.000], mean observation: 166.481 [18.000, 255.000], loss: 0.085827, mean_absolute_error: 51.337396, mean_q: 60.014286, mean_eps: 0.100000
 1311362/2000000: episode: 4036, duration: 20.464s, episode steps: 468, steps per second: 23, episode reward: 435.500, mean reward: 0.931 [0.500, 1.000], mean action: 3.564 [0.000, 6.000], mean observation: 166.052 [13.000, 255.000], loss: 0.088307, mean_absolute_error: 51.156361, mean_q: 59.811875, mean_eps: 0.100000
 1311832/2000000: episode: 4037, duratio

### continue training 

In [2]:
# weights_filename = join(log_dir,'dqn_{}_weights_{}.h5f'.format('AirSimCarRL', 500000))
# dqn.load_weights(weights_filename)
# dqn.test(env, nb_episodes=10, visualize=True)

### testing

In [ ]:
log_dir = 'logs'
if not exists(log_dir):
    os.makedirs(log_dir)
    
weights_filename = join(log_dir,'dqn_{}_weights_{}.h5f'.format('AirSimCarRL', 1000000))
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=True)